# IMAGE CRAWLER

In [26]:
import requests as req
from bs4 import *

In [27]:
url = "https://www.airbnb.co.uk/s/Ljubljana--Slovenia/homes?tab_id=home_tab&refinement_paths%5B%5D=%2Fhomes&query=Ljubljana%2C%20Slovenia&place_id=ChIJ0YaYlvUxZUcRIOw_ghz4AAQ&checkin=2020-11-01&checkout=2020-11-08&source=structured_search_input_header&search_type=autocomplete_click"

In [28]:
## req will send the request to this link and get the data in text format from this link webpage
request = req.get(url)

In [29]:
## we have this text format of webpage from req.get now we want to parse it in html format. 
## once we get the html format then only we could get the image links
soup = BeautifulSoup(request.text, "html.parser") ## this will change text into html format
hrefs = soup.find_all(href=True)
images = soup.find_all('img')

In [30]:
## grab link from all_img_links and append into links list
links = []
for img_link in images:
    links.append(img_link['src'])
links

['https://a0.muscache.com/im/pictures/c12d27de-d2a0-485c-b70a-a65648f7b596.jpg?im_w=720',
 'https://a0.muscache.com/im/pictures/f5d481b3-5451-4e7e-bf70-9efef4ace583.jpg?im_w=720',
 'https://a0.muscache.com/im/pictures/d3553d7c-97a2-487e-907f-a238eaaaba39.jpg?im_w=720',
 'https://a0.muscache.com/im/pictures/59738627-a049-4cec-9e21-2f31302c011c.jpg?im_w=720',
 'https://a0.muscache.com/im/pictures/b53d6945-24a8-4ebd-bd37-9f16f0720c75.jpg?im_w=720',
 'https://a0.muscache.com/im/pictures/miso/Hosting-30003240/original/b5572755-b52d-41fc-ba02-bea5782d2f06.png?im_w=720',
 'https://a0.muscache.com/im/pictures/ed76f499-b27c-4f82-bb96-7ff540534a22.jpg?im_w=720',
 'https://a0.muscache.com/im/pictures/68ef0fea-4760-45bd-9fa0-1ee390e10698.jpg?im_w=720',
 'https://a0.muscache.com/im/pictures/59067ed4-8756-4450-8604-e079ea533450.jpg?im_w=720',
 'https://a0.muscache.com/im/pictures/fe9b86a1-510e-42b3-a9f0-101d47588249.jpg?im_w=720',
 'https://a0.muscache.com/im/pictures/bc3d3c23-f226-47fc-94fa-ad92ae9

In [51]:
## grabbing all the links including the image links
for href in hrefs:
    print(href["href"])

/opensearch.xml
/manifest.json
https://a0.muscache.com/airbnb/static/icons/apple-touch-icon-76x76-3b313d93b1b5823293524b9764352ac9.png
https://a0.muscache.com/airbnb/static/icons/apple-touch-icon-76x76-3b313d93b1b5823293524b9764352ac9.png
https://a0.muscache.com/airbnb/static/icons/apple-touch-icon-120x120-52b1adb4fe3a8f825fc4b143de12ea4b.png
https://a0.muscache.com/airbnb/static/icons/apple-touch-icon-152x152-7b7c6444b63d8b6ebad9dae7169e5ed6.png
https://a0.muscache.com/airbnb/static/icons/apple-touch-icon-180x180-bcbe0e3960cd084eb8eaf1353cf3c730.png
https://a0.muscache.com/airbnb/static/icons/android-icon-192x192-c0465f9f0380893768972a31a614b670.png
https://a0.muscache.com/airbnb/static/logotype_favicon-21cc8e6c6a2cca43f061d2dcabdf6e58.ico
https://a0.muscache.com/airbnb/static/icons/airbnb-0611901eac33ccfa5e93d793a2e21f09.svg
https://a0.muscache.com/airbnb/static/packages/1940-43c866ca.js
https://a0.muscache.com/airbnb/static/packages/niobe-minimalist-client-973063fd.js
https://a0.mus

In [60]:
## function to get img links
def parse_url(url):
    request = req.get(url)
    soup = BeautifulSoup(request.text, "html.parser")
    hrefs = soup.find_all(href=True)
    images = soup.find_all('img')
    links = [link["src"] for link in images]
    return links, hrefs
    

In [65]:
links, hrefs = parse_url("http://4chan.org/")
links

['//s.4cdn.org/image/fp/logo-transparent.png',
 '//i.4cdn.org/a/1619598825449s.jpg',
 '//i.4cdn.org/vg/1618965877392s.jpg',
 '//i.4cdn.org/int/1619628245527s.jpg',
 '//i.4cdn.org/biz/1619631533781s.jpg',
 '//i.4cdn.org/g/1619524010778s.jpg',
 '//i.4cdn.org/mlp/1619588691900s.jpg',
 '//i.4cdn.org/jp/1619369151073s.jpg',
 '//i.4cdn.org/v/1619625817902s.jpg']

In [66]:
href_links = []
for href in hrefs:
    prepare_url(href["href"])
    href_links.append(href["href"])
href_links

['//s.4cdn.org/css/frontpage.12.css',
 '//s.4cdn.org/image/favicon.ico',
 '//s.4cdn.org/image/apple-touch-icon-iphone.png',
 '//s.4cdn.org/image/apple-touch-icon-ipad.png',
 '//s.4cdn.org/image/apple-touch-icon-iphone-retina.png',
 '//s.4cdn.org/image/apple-touch-icon-ipad-retina.png',
 '//www.4chan.org/',
 '#',
 '//boards.4channel.org/a/',
 '//boards.4channel.org/c/',
 '//boards.4channel.org/w/',
 '//boards.4channel.org/m/',
 '//boards.4channel.org/cgl/',
 '//boards.4channel.org/cm/',
 '//boards.4chan.org/f/',
 '//boards.4channel.org/n/',
 '//boards.4channel.org/jp/',
 '//boards.4channel.org/vt/',
 '//boards.4channel.org/v/',
 '//boards.4channel.org/vg/',
 '//boards.4channel.org/vm/',
 '//boards.4channel.org/vmg/',
 '//boards.4channel.org/vp/',
 '//boards.4channel.org/vr/',
 '//boards.4channel.org/vrpg/',
 '//boards.4channel.org/vst/',
 '//boards.4channel.org/co/',
 '//boards.4channel.org/g/',
 '//boards.4channel.org/tv/',
 '//boards.4channel.org/k/',
 '//boards.4channel.org/o/',
 '//

In [67]:
def prepare_url(url):
    import re
    if re.match(r'^[\/]{2}.*', url):
        return f'http:{url}'
    elif re.match(r'^\/\w+.*', url):
        return f'{url}'
    return url

In [68]:
prepare_url('https://a0.muscache.com/airbnb/static/icons/apple-touch-icon-76x76-3b313d93b1b5823293524b9764352ac9.png')

'https://a0.muscache.com/airbnb/static/icons/apple-touch-icon-76x76-3b313d93b1b5823293524b9764352ac9.png'